# Install Required Modules

In [206]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.8 MB/s eta 0:00:00


In [202]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 17.1 MB/s eta 0:00:00


In [1]:
! pip install bs4 lxml kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=a029a429cdffd1e9b8623012c41d0a7ee445540bb04449300aa266ad6745cdac
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


# Download Dataset from Kaggle

In [2]:
import os
os.environ['KAGGLE_USERNAME'] = 'bilalyousaf0014'
os.environ['KAGGLE_KEY'] = '11031bc21c5e3ec23585dbe17dc4267d'

In [3]:
!kaggle datasets download -d bilalyousaf0014/ml-engineer-assessment-dataset

 97% 76.0M/78.6M [00:03<00:00, 27.6MB/s]
100% 78.6M/78.6M [00:03<00:00, 23.4MB/s]


In [4]:
! unzip /content/ml-engineer-assessment-dataset.zip

Archive:  /content/ml-engineer-assessment-dataset.zip
  inflating: assessment_dataset/images/00001.jpeg  
  inflating: assessment_dataset/images/00008.jpeg  
  inflating: assessment_dataset/images/00017.jpeg  
  inflating: assessment_dataset/images/00022.jpeg  
  inflating: assessment_dataset/images/00048.jpeg  
  inflating: assessment_dataset/images/00055.jpeg  
  inflating: assessment_dataset/images/001.jpeg  
  inflating: assessment_dataset/images/1.jpeg  
  inflating: assessment_dataset/images/1001524.jpeg  
  inflating: assessment_dataset/images/1005343.jpeg  
  inflating: assessment_dataset/images/1049854.jpeg  
  inflating: assessment_dataset/images/1072860.jpeg  
  inflating: assessment_dataset/images/1120419.jpeg  
  inflating: assessment_dataset/images/1146885.jpeg  
  inflating: assessment_dataset/images/2.jpeg  
  inflating: assessment_dataset/images/3.jpeg  
  inflating: assessment_dataset/images/4.jpeg  
  inflating: assessment_dataset/images/5.jpeg  
  inflating: assessm

# MODEL IMPLEMENTATION:

In [210]:
import os
import torch
import torch.nn as nn
import numpy as np
import cv2
import pytorch_lightning as pl

from torchvision.models import resnet18, ResNet18_Weights

In [213]:
#@title
class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()
    pretrained_model = resnet18(weights=ResNet18_Weights.DEFAULT)
    self.backbone = nn.Sequential(*list(pretrained_model.children())[:-2])
    self.fc1 = nn.Linear(512,256)
    self.fc2 = nn.Linear(256,256)
    self.fc3 = nn.Linear(256,256)
    self.output = nn.Linear(256,5) # 4 bbox points, 1 specie class

    # ### Initialize the required Layers
    # self.have_object = None
    # self.cat_or_dog = None
    # self.specie = None
    # self.bbox = None
    # ### Initialize the required Layers

  def forward(self, input):
    
    #backbone
    out_backbone = self.backbone(input)
    
    # heads
    x = nn.ReLU()(self.fc1(out_backbone))
    x = nn.ReLU()(self.fc2(x))
    x = nn.ReLU()(self.fc3(x))
    logits = nn.Softmax()(self.output(x))

    return logits

    # return {
    #     "bbox": None,
    #     "object": None,
    #     "cat_or_dog": None,
    #     "specie": None
    # }


In [345]:
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class Model(pl.LightningModule):
    
    def __init__(self):
        super(Model, self).__init__()
        pretrained_model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.backbone = nn.Sequential(*list(pretrained_model.children())[:-2])
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(25088,256)
        self.fc2 = nn.Linear(256,256)
        self.fc3 = nn.Linear(256,256)
        self.output = nn.Linear(256,5) # 4 bbox points, 1 specie class

        ### Initialize the required Layers
        self.specie = 0
        self.xmin = 0
        self.ymin = 0
        self.xmax = 0
        self.xmax = 0
        
    def forward(self, input):

        # backbone
        out_backbone = self.backbone(input)

        # flatten
        x = self.flat(out_backbone)
        
        # heads
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = nn.ReLU()(self.fc3(x))
        pred = nn.Softmax()(self.output(x))

        return pred
    
    def training_step(self, batch, batch_idx):
        image, labels = batch
        predicted_labels = self(image)

        loss_specie = nn.CrossEntropyLoss()(predicted_labels[:, -1], labels[:, -1])
        loss_bbox = nn.MSELoss()(predicted_labels[:, :-1], labels[:, :-1])

        loss = torch.sum(torch.stack([loss_specie, loss_bbox]))
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        image, labels = batch
        predicted_labels = self(image)
        return {}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

# CUSTOM DATALOADER IMPLEMENTATION

In [61]:
train_list = np.load('/content/assessment_dataset/train_list.npy', allow_pickle=True).tolist()
val_list = np.load('/content/assessment_dataset/val_list.npy', allow_pickle=True).tolist()

In [66]:
from bs4 import BeautifulSoup

def read_xml_file(path):
  
    with open(path, 'r') as f:
        data = f.read()
    bs_data = BeautifulSoup(data, 'xml')
    return {
        "cat_or_dog": bs_data.find("name").text,
        "xmin": int(bs_data.find("xmin").text),
        "ymin": int(bs_data.find("ymin").text),
        "xmax": int(bs_data.find("xmax").text),
        "ymax": int(bs_data.find("ymax").text),
        "specie": "_".join(path.split(os.sep)[-1].split("_")[:-1])
    }

In [123]:
read_xml_file('assessment_dataset/labels/pomeranian_180.xml')

{'cat_or_dog': 'dog',
 'xmin': 28,
 'ymin': 1,
 'xmax': 155,
 'ymax': 143,
 'specie': 'pomeranian'}

In [244]:
test = read_xml_file('assessment_dataset/labels/pomeranian_180.xml')
[float(test['xmin'])]

[28.0]

In [132]:
# list all species for class mapping
print(list(set(['_'.join(i.split('_')[:-1]) for i in os.listdir('assessment_dataset/labels/')])))

['pomeranian', 'Persian', 'beagle', 'Abyssinian', 'american_pit_bull_terrier', 'basset_hound', 'american_bulldog', 'Birman', 'chihuahua']


In [311]:
from PIL import Image

import torchvision.transforms as transforms

class CustomDataset():

  def __init__(self, dataset_path, images_list, train=False):

    self.image_dim = (224, 224)
    self.species_map = {'NA': 0, 'pomeranian':1, 'Persian':2, 'beagle':3, 'Abyssinian':4, 'american_pit_bull_terrier':5, 'basset_hound':6, 'american_bulldog':7, 'Birman':8, 'chihuahua':9}
    self.preprocess = []

    image_folder_path = os.path.join(dataset_path, "images")
    label_folder_path = os.path.join(dataset_path, "labels")

    for path in os.listdir(image_folder_path):
        name = path.split(os.sep)[-1].split(".")[0]

        if name in images_list:
          img_fname = [i for i in os.listdir('assessment_dataset/images/') if name in i][0]
          img_path = os.path.join(image_folder_path, img_fname)

          try:
            xml_path = os.path.join(label_folder_path, name+".xml")
            xml_data = read_xml_file(xml_path)

          except FileNotFoundError:
            xml_data = {'cat_or_dog': 'NA', 'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0, 'specie': 'NA'}

          xml_data['specie'] = self.species_map[xml_data['specie']] # map species class to an integer identifier
          self.preprocess.append([img_path, xml_data])

  def __len__(self):
    return len(self.preprocess)

  def __getitem__(self, index):
    image_path, labels = self.preprocess[index]
    labels = [float(labels['xmin']), float(labels['ymin']), float(labels['xmax']), float(labels['ymax']), float(labels['specie'])]
    image = cv2.imread(image_path)
    image = cv2.resize(image, self.image_dim)
    image = torch.from_numpy(image)
    image = image.permute(2, 0, 1)
    labels = torch.tensor([labels])

    return image.type(torch.cuda.FloatTensor), torch.squeeze(labels)

In [246]:
# training_dataset = CustomDataset("assessment_dataset/", images_list=train_list)
# training_loader = DataLoader(training_dataset, batch_size=4, shuffle=True)

In [247]:
# TEST IMAGE
# from google.colab.patches import cv2_imshow

# path = "assessment_dataset/images/n01877812_wallaby.jpeg"
# img = cv2.imread(path)
# cv2_imshow(img)

In [341]:
from torch import utils

training_dataset = CustomDataset("assessment_dataset/", images_list=train_list)
train_dataloaders = utils.data.DataLoader(training_dataset, batch_size=16, shuffle=True)

val_dataset = CustomDataset("assessment_dataset/", images_list=val_list)
val_dataloaders = utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)

In [261]:
# # TEST DATASET AND DATALOADER
# for imgs, labels in train_dataloaders:
#     print("Batch of images has shape: ", imgs.shape)
#     print("Batch of images has type: ", type(imgs))
#     print("Batch of labels has shape: ", labels.shape)

# TRAINING LOOP IMPLEMENTATION

In [346]:
# Instantiate model object / Load Data
model = Model()

trainer = pl.Trainer(max_epochs=2) #devices=1, accelerator="gpu", 
trainer.fit(model, train_dataloaders=train_dataloaders, val_dataloaders=val_dataloaders)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type       | Params
----------------------------------------
0 | backbone | Sequential | 11.2 M
1 | flat     | Flatten    | 0     
2 | fc1      | Linear     | 6.4 M 
3 | fc2      | Linear     | 65.8 K
4 | fc3      | Linear     | 65.8 K
5 | output   | Linear     | 1.3 K 
----------------------------------------
17.7 M    Trainable params
0         Non-trainable params
17.7 M    Total params
70.929    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

<ipython-input-345-71d1032ef51a>:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = nn.Softmax()(self.output(x))


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.
